## Setup

In [1]:
here::here()

[1] "/home/fmsb/eebg24-networks-project"

In [2]:
library(pacman)
p_load("DESeq2", "pheatmap", "EnhancedVolcano", "biomaRt", "GEOquery", "dplyr", "GENIE3", "here", install = FALSE)

## QC

In [3]:
ppin <- read.delim(here::here("03_outputs", "string_interactions_short.tsv"), sep = "\t")

In [4]:
gcn <- read.csv(here::here("03_outputs", "correlation_mtx_merged_filtered.csv"), row.names = 1)

In [5]:
grn1 <- read.csv(here::here("03_outputs", "GSE89408_tfs_matrix_filtered.csv"), row.names = 1)

In [6]:
grn2 <- read.csv(here::here("03_outputs", "E-MTAB-6141_tfs_matrix_filtered.csv"), row.names = 1)

In [7]:
intersected_genes_rows <- intersect(rownames(grn1), rownames(grn2))
intersected_genes_cols <- intersect(names(grn1), names(grn2))

In [8]:
grn1 <- grn1[intersected_genes_rows,intersected_genes_cols] %>% as.matrix()
grn2 <- grn2[intersected_genes_rows,intersected_genes_cols] %>% as.matrix()

In [9]:
grn <- (grn1 + grn2) / 2

In [10]:
abs_cor_w <- abs(grn)
threshold_w <- quantile(abs_cor_w, probs = 0.75, na.rm = TRUE)
grn <- ifelse(abs_cor_w >= threshold_w, grn, 0)

In [11]:
gwas <- read.csv(here::here("03_outputs", "genelist-Eur-downloaded.csv"), row.names = 1)

In [12]:
gda <- read.delim(here::here("03_outputs", "C0003873_disease_gda_summary_filtered.tsv"), sep = "\t", row.names = 1)

## Preparation

In [13]:
gda <- gda %>% filter(rownames(.) %in% rownames(gcn))
gwas <- gwas %>% filter(rownames(.) %in% rownames(gcn))

In [14]:
grn <- grn %>% as.data.frame()
valid_cols <- intersect(rownames(gcn), colnames(grn))

In [15]:
proteomics_hits <- read.delim(here::here("01_data", "proteomics_hits.txt"), col.names = 0)

In [16]:
ppin <- ppin %>% filter(X.node1 %in% proteomics_hits$X0 | node2 %in% proteomics_hits$X0)

In [17]:
drug_int <- read.delim(here::here("01_data", "interactions.tsv"), sep = "\t")

In [18]:
drug <- drug_int %>% filter(gene_claim_name %in% ppin$X.node1 | gene_claim_name %in% ppin$node2) %>% select(gene_name, interaction_score, drug_name, approved)

In [19]:
drug$interaction_score <- as.numeric(drug$interaction_score)
drug <- drug %>%
  filter(drug[["interaction_score"]] >= quantile(drug[["interaction_score"]], 0.75, na.rm = TRUE))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [54]:
gda <- gda %>% tibble::rownames_to_column("genes") %>%
  mutate(source = paste0("(6) ", genes)) %>%
  mutate(target = paste0("(3) ", genes))

In [63]:
gda <- gda %>% rename(source_type = type_source)

In [53]:
gwas <- gwas %>% tibble::rownames_to_column("genes") %>%
  mutate(source = paste0("(5) ", genes)) %>%
  mutate(target = paste0("(3) ", genes))

In [52]:
grn <- which(grn != 0, arr.ind = TRUE) %>%
  as.data.frame() %>%
  dplyr::mutate(
    source = paste0("(4) ", rownames(grn)[row]),
    target = paste0("(3) ", colnames(grn)[col])
  ) %>%
  dplyr::select(source, target) %>%
  mutate(source_type = "grn")

In [51]:
gcn <- which(gcn != 0, arr.ind = TRUE) %>%
  as.data.frame() %>%
  dplyr::mutate(
    source = paste0("(3) ", rownames(gcn)[row]),
    target = paste0("(3) ", colnames(gcn)[col])
  ) %>%
  dplyr::select(source, target) %>%
  mutate(source_type = "gcn")

In [42]:
ppin <- ppin %>% mutate(source = paste0("(2) ", X.node1), target = paste0("(2) ", node2), source_type = "ppin") %>% select(source, target, source_type)


ppin_source_extended <- ppin %>%
  mutate(source = gsub("\\(2\\)", "(3)", source))

ppin_target_extended <- ppin %>%
  mutate(target = gsub("\\(2\\)", "(3)", target))

ppin <- bind_rows(ppin, ppin_source_extended, ppin_target_extended)

In [50]:
drug <- drug %>% rename(source = drug_name, target = gene_name) %>% mutate(source_type = "drug") %>%
        mutate(target = paste0("(2) ", target)) %>% select(source, target, source_type, approved)

In [61]:
gda

genes,Score_gda,type_source,source,target
<chr>,<dbl>,<chr>,<chr>,<chr>
FCGR2A,0.5,gda,(6) FCGR2A,(3) FCGR2A
IL2RA,0.5,gda,(6) IL2RA,(3) IL2RA
IL10,0.5,gda,(6) IL10,(3) IL10


In [70]:
ntx_df <- bind_rows(grn, gcn, ppin, gwas, drug, gda) %>% select(-genes, -Score_gda) 

In [72]:
ntx_df$source_type = as.factor(ntx_df$source_type)

In [73]:
summary(ntx_df)

    source             target          source_type     approved        
 Length:187120      Length:187120      drug:   448   Length:187120     
 Class :character   Class :character   gcn :152882   Class :character  
 Mode  :character   Mode  :character   gda :     3   Mode  :character  
                                       grn : 32897                     
                                       gwas:    14                     
                                       ppin:   876                     

# Export

In [89]:
write.csv(drug, here("03_outputs", "network-ready-layers", "drug.csv"))
write.csv(ppin, here("03_outputs", "network-ready-layers", "ppin.csv"))
write.csv(gcn, here("03_outputs", "network-ready-layers", "gcn.csv"))
write.csv(grn, here("03_outputs", "network-ready-layers", "grn.csv"))
write.csv(gwas, here("03_outputs", "network-ready-layers", "gwas.csv"))
write.csv(gda, here("03_outputs", "network-ready-layers", "gda.csv"))

In [75]:
write.csv(ntx_df, here("03_outputs", "network-ready-layers", "network_dataframe.csv"), quote = FALSE, row.names=FALSE)